In [1]:
import os
import json
import pendulum
import pandas as pd

from tqdm import tqdm

from twarc.client2 import Twarc2
from twarc.expansions import TWEET_FIELDS
from twarc.expansions import ensure_flattened

In [2]:
from helpers.clean_tweets import clean_tweets_dataset
from helpers.get_n_grams import get_n_grams

In [3]:
CREDENTIALS_TWITTER_API = {
    'bearer_token': "Enter your own Bearer token",

    # 'api_key': "Enter your own API Key",
    # 'api_secret_key': "Enter your own API Secret Key",
    # 'access_token': "Enter your own access_token",
    # 'access_token_secret': "Enter your own access_token_secret"
}

IS_ACADEMIC = False # Cambiar a True, si las credenciales son Academicas

In [5]:
def get_tweets(credentials_api,
               queries_list,
               output_file,
               since_date, until_date,
               is_academic=False):
    """Function in charge of scrape tweets from the 
    official Twitter API, using the library named Twarc.

    Args:
        credentials_api (dict): Dictionary with the Twitter API credentials.
        queries_list (list[str]): List of queries to scrape.
        output_file (str): Path to the file in which to store the results.
        since_date (datetime): Start of the time span to scrape.
        until_date (datetime): End of the time span to scrape.
        is_academic (bool, optional): If the credentials has Research 
                                      Academic access level.
    """

    # Instiate the Twarc Client
    twarc_client = Twarc2(**credentials_api)

    # Make some tweaks for using the research credentials
    max_size = 100
    tweet_fields = TWEET_FIELDS.copy()
    search_func = twarc_client.search_recent
    if(is_academic):
        search_func = twarc_client.search_all
        max_size = 500

        # Remove the context_annotations attr to
        # scrape 500 tweets per request
        tweet_fields.remove('context_annotations')

    tweet_fields = ','.join(tweet_fields)

    with open(output_file, 'a') as pages_file:
        for query in tqdm(queries_list):

            search_results = search_func(query=query,
                                         start_time=since_date,
                                         end_time=until_date,
                                         tweet_fields=tweet_fields,
                                         max_results=max_size)

            # Write all the obtained tweets
            for page in search_results:

                # Write one by one the tweets
                for tweet in ensure_flattened(page):
                    json.dump(tweet, pages_file, ensure_ascii=False)
                    pages_file.write('\n')

# Definir Queries Útiles

En esta notebook vamos a revisar una propuesta del proceso que se puede seguir para definir queries que ayuden a obtener un volumen suficiente de tweets constantemente. Se usará de ejemplo el proceso realizado para el proyecto del Laboratorio de Migración.

Para ello, se veran los siguientes puntos:
- Queries para un primer dataset
- N-Grams para expandir dataset
- Proceso iterativo
- Añadir operadores de región y lenguaje

Nota: Para este proceso nos interesa obtener un volumen considerable de tweets, por lo que entre más días se puedan minar, mejor. En caso de contar con acceso [*Essential* o *Elevated*](https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-level) se recomienda [aplicar de manera gratuita a credenciales Académicas](https://developer.twitter.com/en/products/twitter-api/academic-research). 

TO-DO: Precedente de credenciales academicas

## Primer Dataset

### Identificar países y lenguajes de interés
Un primer paso es identificar en que lenguaje vamos a minar, en el caso del Laboratorio de Migración se minan tweets de los países hispanohablantes de LATAM (español), países del caribe (inglés) y Brasil (portugués). Dependiendo de que tanto se habla ese idioma en el mundo y los países que nos interesa cubrir, van a cambiar que tan específicos o generales pueden que ser los queries y qué operadores usaríamos.

### Ejemplos proyecto Laboratorio de Migración

### Inglés
Tweets en inglés se publican desde todas partes del mundo y en este caso, solo nos interesan ciertos países que por densidad de población no pueden generar un gran volumen de tweets. Por lo que si no usamos el operador `place_country`, vamos a obtener tweets de todo el mundo y eso significaría invertir más tiempo y recursos en el minado, procesamiento, etc. Para evitar esto, desde ya se puede definir que usaremos el operador `place_country` para así limitar la búsqueda de tweets a esos países y esto nos dará la oportunidad de usar keywords con un alcance más abierto. 

En el ejemplo de migración se podrían usar: `migrants, immigrants, emigrants, migration, immigration, emigration, migratory, immigratory y emigratory` y luego añadir variaciones como `pro-migrants, anti-migrants, ...`

El query podria ser: `(migrants OR immigrants OR emigrants OR migration OR immigration OR emigration OR migratory OR immigratory OR emigratory) (place_country:BB OR place_country:GY OR place_country:BZ OR place_country:JM OR place_country:HT OR place_country:SR)`

### Portugués
En este caso, los tweets en portugués van a venir de Brasil o de Portugal principalmente, por lo que se podría aplicar el operador `place_country` para obtener solo tweets publicados desde Brasil o minar queries sin ninguna restricción para luego filtrarlos basándose en sí son geo-localizables a otro lado que no sea Brasil. 

Palabras claves podrian ser: `migratórias, migração, migrações, migrantes, migradas`

El query podria ser: `(migratórias OR migração OR migrações OR migrantes OR migradas) place_country:BR`

### Español
Para este lenguaje, exceptuando a España, nos interesa obtener tweets del conjunto que van a generar la mayoria de tweets en español. Y aunque es buena práctica utilizar queries más generales junto con el operador `place_country`, esto nos dejaría con un volumen muy inferior y que no terminaría de reflejar las conversaciones reales de lo que se podría obtener.

Por lo que aquí conviene hacer ambas cosas, tener queries con keywords generales, pero cerrado a solo los países de interés y tener queries con keywords más específicos, pero sin ninguna limitación de geolocalización, para luego tratar de extraer el país de origen por otros atributos (Esto está en la siguiente notebook).

Un query ejemplo podría ser: `(migrar OR inmigrar OR emigrar OR migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes OR migratorios OR inmigratorios OR emigratorios OR migratorias OR inmigratorias OR emigratorias) (place_country:MX OR place_country:AR OR place_country:AR)`. Esto se replicaría para todos los países que nos interesa y nos serviría como primer dataset para iterar sobre los queries. No solo para los queires con el filtro por pais, tambien para aquellos que no lo tienen, ya que esos debe de ser más especificos para no traernos demasiado ruido pero que cubran la mayor parate de la conversación para así tener un volumen suficiente.

Para esta notebook usaremos de ejemplo la definición de queries en español, y como se mencionó previamente, el primer paso es definir una lista preliminar de keywords y concatenarlas con el operador `OR`. Optar por palabras que de manera explícita hable del tema de interés.

In [6]:
list_keywords = ["migrar", "inmigrar", "emigrar", 
                 "migrante", "inmigrante", "emigrante", 
                 "migrantes", "inmigrantes", "emigrantes", 
                 "migratorios", "inmigratorios", "emigratorios", 
                 "migratorias", "inmigratorias", "emigratorias"]
list_keywords = ' OR '.join(list_keywords)

En seguida hay que añadir los operadores del pais de interes. Esto nos ayudara a encontrar keywords o expresiones que esten atadas al contexto de cada pais, ayudando asi a filtrar un poco la conversación migratoria a los paises de interes.

In [7]:
lst_queries = f"({list_keywords}) (place_country:MX OR place_country:AR OR place_country:CO)"
lst_queries

'(migrar OR inmigrar OR emigrar OR migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes OR migratorios OR inmigratorios OR emigratorios OR migratorias OR inmigratorias OR emigratorias) (place_country:MX OR place_country:AR OR place_country:CO)'

Definimos un intervalo de tiempo considerable para obtener un volumen útil de tweets. En el proyecto del Laboratorio de Migración se empezó con minar tweets de todo el año 2019 y 2020, al ser tanto tiempo, podremos obtener tweets de momentos en los que ocurrieran eventos relacionados a migración, así como en momentos donde sea un tema menos viral. 

Para este ejemplo usaremos todo lo que va del 2022.

Nota: Todos los archivos que se obtengan durante el desarrollo de esta notebook se van a encontrar en el directorio "./archivos_queries"

In [8]:
lst_queries = [lst_queries]

date_start = pendulum.datetime(year=2022, month=1, day=1)
date_end = pendulum.datetime(year=2022, month=9, day=1)

file_tweets = os.path.abspath("./archivos_queries/1_dataset.jsonl")

Se ejecuta la función para minar los tweets

In [9]:
get_tweets(credentials_api = CREDENTIALS_TWITTER_API,
           queries_list = lst_queries,
           output_file = file_tweets,
           since_date = date_start, 
           until_date = date_end,
           is_academic= IS_ACADEMIC)

100%|██████████| 1/1 [01:25<00:00, 85.51s/it]


Vemos cuantos tweets obtuvimos

In [10]:
!wc -l ./archivos_queries/1_dataset.jsonl

8584 ./archivos_queries/1_dataset.jsonl


Estos 8,590 tweets nos van a servir como primer dataset para empezar a identificar keywords que esten relacionadas con el fenomeno que queremos estudiar, ya sea que se refieran a el de manera :
- Explicita como lo es decir "migrante" o "inmigrantes" 
- Implicito como "los mexicanos en estado unidos" o "las personas ilegales".

Para lograr hacer esto primero hay que cargar los identificadores unicos (IDs) y textos de los tweets. 

In [11]:
df_tweets = pd.read_json("./archivos_queries/1_dataset.jsonl", lines=True)
df_tweets = df_tweets[['id', 'text']]
df_tweets

,id,text
0,1565119364566847488,En el Centro de Jubilados Un Sueño de B° La Es...
1,1565104878166446080,"El Grupo de Asuntos Étnicos, Migrantes y Refug..."
2,1565103949337837568,Lanzamiento de #INTEGRATE de @alcaldiabogota @...
3,1565102572586909696,"@AntaresVazAla Por supuesto, plática del foro ..."
4,1565095116074876928,Por fin entendieron que los nietos tenemos el ...
...,...,...
8579,1477088691877261312,"Sin duda, fue un año de mucho trabajo. Y nos p..."
8580,1477088689901682688,💇‍♀️ Servimos de escuela para capacitar a 125 ...
8581,1477086472469684224,@GSandovalSalas @DIF_NMX @GobiernoMX @SRE_mx @...
8582,1477067319830478848,@denadastotales @alfiemart Pero si con ese mod...


### N-Grams

Estas son un concepto muy utilizado en el procesamiento de lenguaje natural (NLP, por siglas en ingles), y se pueden entender como una secuencia de N palabras, por ejemplo:
- Dream on (2-Grams)
- Kickstart My Heart (3-Grams)
- Hail to the King (4-Grams)

Estas secuencias de texto se pueden contabilizar en un corpus de texto y obtener la frecuencia de aparición de una palabra tras la otra, y así saber qué palabras puede considerarse como una sola entidad. Por ejemplo, en un texto sobre la historia de la banda de rock Aerosmith el 2-gram "Dream On" se puede considerar una sola palabra, dado que se refiere a la canción que sirvió como soundtrack de la pelicula  Armagedon. También puede servir para realizar una predicción de que palabras siguen, como el autocompletado que realiza Spotify para mostrar una lista de opciones de la canción original y covers cuando solo escribimos "Kick Start...".

Podemos considerar los N-Grams como una herramienta util para encontrar patrones entre los tweets que hablan sobre migración y así extender la lista de keywords y/o formar queries que se apoyen en que un tweet contenga al menos dos keywords.

Primero hay que preprocesar los tweets para así eliminar partes del texto que no sean de utilidad y pasar todo a minusculas, por ejemplo:
- URLs
- Numeros
- Emojis
- Caracteres especiales
- Risas (Jajaj o Hahaha)
- Espacios en blanco multiples
- Mencion a usuarios
- Borrar stopwords (que, como, cual, etc)

In [12]:
df_tweets = clean_tweets_dataset(df_tweets, doc_column='text', clean_col='text_preprocessed')
df_tweets

,id,text,text_preprocessed
0,1565119364566847488,En el Centro de Jubilados Un Sueño de B° La Es...,centro jubilados sueno estrella pudimos compar...
1,1565104878166446080,"El Grupo de Asuntos Étnicos, Migrantes y Refug...",grupo asuntos etnicos migrantes refugiados per...
2,1565103949337837568,Lanzamiento de #INTEGRATE de @alcaldiabogota @...,lanzamiento integrate brindara atencion migran...
3,1565102572586909696,"@AntaresVazAla Por supuesto, plática del foro ...",platica foro sao paulo socialismo cuba venezue...
4,1565095116074876928,Por fin entendieron que los nietos tenemos el ...,entendieron nietos derecho adoptar doble ciuda...
...,...,...,...
8579,1477088691877261312,"Sin duda, fue un año de mucho trabajo. Y nos p...",duda ano preparamos continuaremos luchando der...
8580,1477088689901682688,💇‍♀️ Servimos de escuela para capacitar a 125 ...,servimos escuela capacitar migrantes peluqueri...
8581,1477086472469684224,@GSandovalSalas @DIF_NMX @GobiernoMX @SRE_mx @...,culpables inmigrantes muertos chiapa cirzo bur...
8582,1477067319830478848,@denadastotales @alfiemart Pero si con ese mod...,modelo rico top pib per capital destino millon...


In [13]:
srs_ngrams = get_n_grams(df_tweets['text_preprocessed'])
srs_ngrams

/home/noecampos/.pyenv/versions/BID/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,n_gram,total_freq
0,poblacion migrante,243
1,migrantes venezolanos,229
2,personas migrantes,136
3,prov inmigrante,130
4,brenas capital,129
...,...,...
95,etnicos migrantes,25
96,frontera sur,25
97,migrantes centroamericanos,25
98,temas migratorios,25


In [17]:
srs_ngrams[60:]

,n_gram,total_freq
60,unico migrantes,42
61,ninos ninas,40
62,foto brenas,38
63,migrantes refugiadas,38
64,miles migrantes,36
65,ninos migrantes,36
66,venezolanos rumv,35
67,permiso proteccion,34
68,antonio texas,31
69,atencion poblacion,31
